In [11]:
import os
import json
import uuid
from typing import Dict, Any, Optional, List
from datetime import datetime

# Pydantic models for structured data
from pydantic import BaseModel, Field, field_validator
import google.generativeai as genai
import requests
from sympy import symbols, sympify, diff, integrate

# Modern LangChain imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.messages import HumanMessage, AIMessage
from langchain import hub

In [12]:
# ============================================================================
# PYDANTIC PARSERS (Task 1: IMPLEMENT PARSERS)
# ============================================================================

class WeatherRequest(BaseModel):
    """Parser for weather requests"""
    city: str = Field(..., min_length=1, description="City name for weather query")
    units: Optional[str] = Field("metric", description="Temperature units (metric/imperial)")
    
    @field_validator('city')
    @classmethod
    def validate_city(cls, v):
        if not v.strip():
            raise ValueError("City name cannot be empty")
        return v.strip().title()

class MathExpression(BaseModel):
    """Parser for mathematical expressions"""
    expression: str = Field(..., description="Mathematical expression to compute")
    operation: str = Field(..., description="Type of operation: derivative or integral")
    variable: str = Field("x", description="Variable with respect to which to compute")
    
    @field_validator('operation')
    @classmethod
    def validate_operation(cls, v):
        if v.lower() not in ['derivative', 'integral']:
            raise ValueError("Operation must be 'derivative' or 'integral'")
        return v.lower()

class ConversationContext(BaseModel):
    """Parser for conversation context"""
    user_input: str
    session_id: str
    timestamp: datetime = Field(default_factory=datetime.now)
    intent: Optional[str] = None
    
class AgentResponse(BaseModel):
    """Structured response from agent"""
    content: str
    tool_used: Optional[str] = None
    confidence: Optional[float] = Field(None, ge=0.0, le=1.0)
    metadata: Dict[str, Any] = Field(default_factory=dict)

In [14]:
# ============================================================================
# API SETUP
# ============================================================================

# Load Google API key
with open("Google_API_Key.txt") as f:
    google_api_key = f.read().strip()
genai.configure(api_key=google_api_key)
os.environ["GOOGLE_API_KEY"] = google_api_key

# Load OpenWeather API key
with open("Weather_API_Key.txt", "r") as file:
    OPENWEATHER_API_KEY = file.read().strip()

# Load LangSmith API key
with open("LangSmith_API_Key.txt", "r") as f:
    langsmith_key = f.read().strip()
os.environ["LANGSMITH_API_KEY"] = langsmith_key
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "Trial_agent_c"

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-pro", temperature=0)

FileNotFoundError: [Errno 2] No such file or directory: 'LangSmith_API_Key.txt'

In [5]:
# ============================================================================
# TOOLS WITH PYDANTIC VALIDATION
# ============================================================================

@tool
def compute_derivative(expression: str) -> str:
    """Compute the derivative of a mathematical expression with respect to x."""
    try:
        # Parse with Pydantic
        math_expr = MathExpression(expression=expression, operation="derivative")
        
        x = symbols(math_expr.variable)
        expr = sympify(math_expr.expression)
        derivative = diff(expr, x)
        
        return f"The derivative of {math_expr.expression} is: {derivative}"
    except Exception as e:
        return f"Error computing derivative: {e}"

@tool
def compute_integral(expression: str) -> str:
    """Compute the indefinite integral of a mathematical expression with respect to x."""
    try:
        # Parse with Pydantic
        math_expr = MathExpression(expression=expression, operation="integral")
        
        x = symbols(math_expr.variable)
        expr = sympify(math_expr.expression)
        integral = integrate(expr, x)
        
        return f"The integral of {math_expr.expression} is: {integral} + C"
    except Exception as e:
        return f"Error computing integral: {e}"

@tool
def weather_tool(city: str) -> str:
    """Get current weather information for a specified city."""
    try:
        # Parse with Pydantic
        weather_req = WeatherRequest(city=city)
        
        base_url = "http://api.openweathermap.org/data/2.5/weather"
        params = {
            "q": weather_req.city,
            "appid": OPENWEATHER_API_KEY,
            "units": weather_req.units
        }
        
        response = requests.get(base_url, params=params)
        
        if response.status_code != 200:
            return f"Sorry, couldn't fetch weather for {weather_req.city}."
            
        data = response.json()
        weather_desc = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        wind_speed = data["wind"]["speed"]
        
        return (f"Weather in {weather_req.city}: {weather_desc}, "
                f"temperature {temp}°C, humidity {humidity}%, "
                f"wind speed {wind_speed} m/s.")
                
    except Exception as e:
        return f"Error fetching weather: {e}"

@tool
def general_chat(query: str) -> str:
    """Handle general conversation and questions."""
    try:
        # Parse conversation context
        context = ConversationContext(
            user_input=query,
            session_id="default",  # You can make this dynamic
            intent="general_chat"
        )
        
        response = llm.invoke(context.user_input)
        return response.content
    except Exception as e:
        return f"Error in general chat: {e}"


In [6]:
# ============================================================================
# CONDITIONAL ROUTING (Task 2: CONDITIONAL NODES AND EDGES)
# ============================================================================

class RouteDecision(BaseModel):
    """Decision model for routing user queries"""
    intent: str = Field(..., description="Detected intent: weather, math, or general")
    confidence: float = Field(..., ge=0.0, le=1.0, description="Confidence in routing decision")
    tool_name: str = Field(..., description="Tool to use for this intent")
    
    @field_validator('intent')
    @classmethod
    def validate_intent(cls, v):
        valid_intents = ['weather', 'math_derivative', 'math_integral', 'general']
        if v not in valid_intents:
            raise ValueError(f"Intent must be one of {valid_intents}")
        return v

def route_query(query: str) -> RouteDecision:
    """Conditional router that decides which tool to use based on query analysis."""
    query_lower = query.lower()
    
    # Weather routing
    weather_keywords = ['weather', 'temperature', 'rain', 'sunny', 'cloudy', 'forecast']
    if any(keyword in query_lower for keyword in weather_keywords):
        return RouteDecision(
            intent="weather",
            confidence=0.9,
            tool_name="weather_tool"
        )
    
    # Math routing
    derivative_keywords = ['derivative', 'differentiate', 'diff', "d/dx"]
    integral_keywords = ['integral', 'integrate', 'antiderivative']
    
    if any(keyword in query_lower for keyword in derivative_keywords):
        return RouteDecision(
            intent="math_derivative",
            confidence=0.85,
            tool_name="compute_derivative"
        )
    
    if any(keyword in query_lower for keyword in integral_keywords):
        return RouteDecision(
            intent="math_integral",
            confidence=0.85,
            tool_name="compute_integral"
        )
    
    # Default to general chat
    return RouteDecision(
        intent="general",
        confidence=0.7,
        tool_name="general_chat"
    )


In [7]:
# ============================================================================
# MODERN AGENT SETUP WITH MEMORY
# ============================================================================

# Create tools list
tools = [compute_derivative, compute_integral, weather_tool, general_chat]

# Get the React prompt template
prompt = hub.pull("hwchase17/react")

# Create the agent
agent = create_react_agent(llm, tools, prompt)

# Create agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)

# Setup message history (modern approach)
store: Dict[str, BaseChatMessageHistory] = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Wrap agent with message history
conversational_agent = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

c:\Users\Sebastian\AppData\Local\Programs\Python\Python313\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
# ============================================================================
# MAIN CONVERSATION LOOP
# ============================================================================

def run_conversation():
    """Main conversation loop with structured parsing and routing."""
    session_id = str(uuid.uuid4())
    print("Multi-Tool Agent Ready! (Type 'exit' to quit)")
    print("I can help with: weather, calculus (derivatives/integrals), and general chat")
    
    while True:
        try:
            user_input = input("\nYou: ")
            if user_input.lower() in ["exit", "quit", "q"]:
                print("Bot: Goodbye! Have a great day.")
                break
            
            # Parse user input with Pydantic
            context = ConversationContext(
                user_input=user_input,
                session_id=session_id
            )
            
            # Get routing decision
            route_decision = route_query(user_input)
            print(f"Routing: {route_decision.intent} (confidence: {route_decision.confidence:.2f})")
            
            # Execute with agent
            response = conversational_agent.invoke(
                {"input": user_input},
                config={"configurable": {"session_id": session_id}}
            )
            
            # Parse response with Pydantic
            agent_response = AgentResponse(
                content=response.get("output", ""),
                tool_used=route_decision.tool_name,
                confidence=route_decision.confidence,
                metadata={"session_id": session_id, "timestamp": datetime.now().isoformat()}
            )
            
            print(f"Bot: {agent_response.content}")
            
        except Exception as e:
            print(f"Error: {e}")
            continue

if __name__ == "__main__":
    run_conversation()

Multi-Tool Agent Ready! (Type 'exit' to quit)
I can help with: weather, calculus (derivatives/integrals), and general chat
Routing: weather (confidence: 0.90)


> Entering new AgentExecutor chain...
The user has three requests:
1.  Derive 2X.
2.  Integrate 2X.
3.  Get the weather for Lima.

I will use the `compute_derivative` tool for the first request, the `compute_integral` tool for the second, and the `weather_tool` for the third. I will then combine the results into a single answer.
Action: compute_derivative
Action Input: 2*xThe derivative of 2*x is: 2I have the derivative of 2X. Now I need to compute the integral of 2X and get the weather in Lima. I will proceed with the integration first.
Action: compute_integral
Action Input: 2*xThe integral of 2*x is: x**2 + CI have the derivative and the integral of 2X. Now I need to get the weather in Lima.
Action: weather_tool
Action Input: LimaSorry, couldn't fetch weather for Lima.

> Finished chain.
Bot: Agent stopped due to iteration li